# **Chapter 20: Authentication & Security**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Implement OAuth 2.0 authentication flows and handle JWT tokens securely
- Store sensitive data (tokens, credentials) using platform-specific secure storage
- Integrate biometric authentication (fingerprint/face recognition) in Flutter apps
- Implement SSL/TLS certificate pinning to prevent man-in-the-middle attacks
- Manage API keys and environment-specific configuration securely
- Follow OWASP Mobile Security guidelines for Flutter applications

---

## **Prerequisites**

- Completed Chapter 6: Asynchronous Programming (Futures, Streams, async/await)
- Completed Chapter 19: HTTP Requests & REST APIs
- Understanding of state management (Provider/Riverpod/BLoC) from Chapters 12-15
- Basic understanding of web security concepts (HTTPS, tokens, encryption)
- Access to physical device or emulator with biometric capabilities (for Section 20.3)

---

## **20.1 OAuth 2.0 and JWT Authentication**

OAuth 2.0 is the industry-standard protocol for authorization. In Flutter apps, it's commonly used for "Sign in with Google/Apple/Facebook" functionality. JSON Web Tokens (JWT) are used to securely transmit information between parties as a JSON object.

### **Understanding OAuth 2.0 Flow**

```dart
// oauth_service.dart
// This file handles the complete OAuth 2.0 authentication flow
// including PKCE (Proof Key for Code Exchange) for security

import 'dart:convert';
import 'dart:math';
import 'package:crypto/crypto.dart';
import 'package:http/http.dart' as http;
import 'package:url_launcher/url_launcher.dart';
import 'package:uni_links/uni_links.dart';

class OAuthService {
  // Configuration constants for OAuth provider
  // In production, these should come from environment variables
  final String clientId;
  final String redirectUri;
  final String authorizationEndpoint;
  final String tokenEndpoint;
  
  // PKCE parameters for secure authorization code flow
  String? _codeVerifier;
  String? _codeChallenge;
  
  OAuthService({
    required this.clientId,
    required this.redirectUri,
    required this.authorizationEndpoint,
    required this.tokenEndpoint,
  });
  
  /// Generates a cryptographically secure code verifier for PKCE
  /// PKCE prevents authorization code interception attacks
  String _generateCodeVerifier() {
    // Create 32 random bytes (256 bits of entropy)
    final random = Random.secure();
    final values = List<int>.generate(32, (i) => random.nextInt(256));
    
    // Base64URL encode without padding
    // This creates a 43-character string suitable for PKCE
    return base64UrlEncode(values).replaceAll('=', '');
  }
  
  /// Generates code challenge from verifier using SHA256
  /// The challenge is sent to the authorization server
  /// The verifier is sent later to exchange for tokens
  String _generateCodeChallenge(String verifier) {
    // Convert verifier string to bytes
    final bytes = utf8.encode(verifier);
    
    // Compute SHA256 hash
    final digest = sha256.convert(bytes);
    
    // Base64URL encode the hash
    return base64UrlEncode(digest.bytes).replaceAll('=', '');
  }
  
  /// Initiates the OAuth 2.0 authorization flow
  /// Opens browser for user authentication
  Future<void> initiateOAuthFlow() async {
    // Generate PKCE parameters
    _codeVerifier = _generateCodeVerifier();
    _codeChallenge = _generateCodeChallenge(_codeVerifier!);
    
    // Build authorization URL with required parameters
    final authUrl = Uri.parse(authorizationEndpoint).replace(
      queryParameters: {
        'response_type': 'code',           // Requesting authorization code
        'client_id': clientId,             // Application identifier
        'redirect_uri': redirectUri,       // Where to redirect after auth
        'code_challenge': _codeChallenge,  // PKCE challenge
        'code_challenge_method': 'S256',   // SHA256 hash method
        'scope': 'openid profile email',   // Requested permissions
        'state': _generateState(),         // CSRF protection
      },
    );
    
    // Launch external browser for authentication
    // Using external browser is more secure than WebView
    if (await canLaunchUrl(authUrl)) {
      await launchUrl(
        authUrl,
        mode: LaunchMode.externalApplication,
      );
    } else {
      throw Exception('Could not launch OAuth URL');
    }
  }
  
  /// Generates random state parameter to prevent CSRF attacks
  String _generateState() {
    final random = Random.secure();
    final values = List<int>.generate(16, (i) => random.nextInt(256));
    return base64UrlEncode(values);
  }
  
  /// Exchanges authorization code for access and refresh tokens
  /// This is called after the user is redirected back to the app
  Future<AuthTokens> exchangeCodeForTokens(String authorizationCode) async {
    // Validate we have the verifier stored
    if (_codeVerifier == null) {
      throw StateError('OAuth flow not initiated');
    }
    
    // Prepare token request body
    final body = {
      'grant_type': 'authorization_code',
      'client_id': clientId,
      'code': authorizationCode,
      'redirect_uri': redirectUri,
      'code_verifier': _codeVerifier,  // PKCE: prove we initiated the flow
    };
    
    // Make token request to OAuth server
    final response = await http.post(
      Uri.parse(tokenEndpoint),
      headers: {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
      },
      body: body,
    );
    
    if (response.statusCode == 200) {
      final data = jsonDecode(response.body);
      
      // Parse and return tokens
      return AuthTokens(
        accessToken: data['access_token'],
        refreshToken: data['refresh_token'],
        idToken: data['id_token'],  // JWT containing user info
        expiresIn: data['expires_in'],
        tokenType: data['token_type'],
      );
    } else {
      throw Exception('Token exchange failed: ${response.body}');
    }
  }
}

/// Data class representing OAuth tokens
/// This should be stored securely (see Section 20.2)
class AuthTokens {
  final String accessToken;       // Short-lived (minutes/hours)
  final String? refreshToken;     // Long-lived (days/months), get new access tokens
  final String? idToken;          // JWT with user identity claims
  final int expiresIn;            // Seconds until access token expires
  final String tokenType;         // Usually "Bearer"
  
  AuthTokens({
    required this.accessToken,
    this.refreshToken,
    this.idToken,
    required this.expiresIn,
    required this.tokenType,
  });
  
  /// Calculate actual expiration time
  DateTime get expirationTime {
    return DateTime.now().add(Duration(seconds: expiresIn));
  }
  
  /// Check if access token is expired or about to expire
  bool get isExpired {
    // Consider token expired 5 minutes before actual expiration
    // to account for network latency and clock skew
    return DateTime.now().isAfter(
      expirationTime.subtract(Duration(minutes: 5)),
    );
  }
}
```

**Explanation:**

- **`Random.secure()`**: Uses cryptographically secure random number generation, essential for security tokens that must be unpredictable.
- **PKCE (Proof Key for Code Exchange)**: The `_codeVerifier` is a secret generated by the client, hashed to create `_codeChallenge`. The challenge is sent to the server, but the verifier is only sent when exchanging the code for tokens. This prevents authorization code interception attacks.
- **`sha256.convert()`**: Creates a SHA-256 hash of the verifier. SHA-256 is a one-way function, so the server can verify the challenge matches without knowing the verifier in advance.
- **`base64UrlEncode`**: Encodes binary data as URL-safe base64 (replacing `+` with `-`, `/` with `_`, and removing `=` padding) so it can be safely transmitted in URLs.
- **`response_type: 'code'`**: Requests an authorization code rather than an implicit token. The authorization code flow is more secure than implicit flow because tokens aren't exposed in URLs.
- **`state` parameter**: A random value that the server returns unchanged. The client verifies it matches to prevent Cross-Site Request Forgery (CSRF) attacks.
- **`LaunchMode.externalApplication`**: Opens the system browser rather than an in-app WebView. This ensures the user sees the actual OAuth provider URL in the address bar, preventing phishing attacks.

### **JWT Token Handling and Validation**

```dart
// jwt_manager.dart
// Handles decoding, validating, and refreshing JWT tokens

import 'dart:convert';
import 'package:flutter/foundation.dart';

/// Manages JWT (JSON Web Token) operations
/// JWT structure: header.payload.signature
class JWTManager {
  /// Decodes a JWT token without verification
  /// WARNING: This does NOT verify the signature!
  /// Signature verification must be done server-side or with public key
  static Map<String, dynamic> decodeToken(String token) {
    try {
      // Split the token into its three parts
      final parts = token.split('.');
      if (parts.length != 3) {
        throw FormatException('Invalid token format');
      }
      
      // Decode the payload (middle part)
      // JWT uses Base64URL encoding, but standard base64 decode works
      // if we handle padding correctly
      String payload = parts[1];
      
      // Add padding if necessary (Base64URL removes padding '=')
      // Base64 length must be multiple of 4
      final padding = 4 - payload.length % 4;
      if (padding != 4) {
        payload += '=' * padding;
      }
      
      // Replace URL-safe characters with standard Base64 characters
      payload = payload.replaceAll('-', '+').replaceAll('_', '/');
      
      // Decode Base64 to bytes, then UTF-8 decode to string
      final decoded = utf8.decode(base64Decode(payload));
      
      // Parse JSON payload
      return jsonDecode(decoded);
    } catch (e) {
      throw FormatException('Failed to decode JWT: $e');
    }
  }
  
  /// Validates token claims
  /// Returns validation result with detailed error information
  static TokenValidationResult validateToken(
    String token, {
    String? expectedIssuer,
    String? expectedAudience,
  }) {
    try {
      final claims = decodeToken(token);
      
      // Check expiration (exp claim - Unix timestamp)
      if (claims.containsKey('exp')) {
        final exp = claims['exp'] as int;
        final expirationDate = DateTime.fromMillisecondsSinceEpoch(exp * 1000);
        
        // Allow 60 seconds clock skew for server time differences
        if (DateTime.now().isAfter(expirationDate.add(Duration(seconds: 60)))) {
          return TokenValidationResult(
            isValid: false,
            error: 'Token expired',
            claims: claims,
          );
        }
      }
      
      // Check not before time (nbf claim)
      if (claims.containsKey('nbf')) {
        final nbf = claims['nbf'] as int;
        final notBefore = DateTime.fromMillisecondsSinceEpoch(nbf * 1000);
        
        if (DateTime.now().isBefore(notBefore)) {
          return TokenValidationResult(
            isValid: false,
            error: 'Token not yet valid',
            claims: claims,
          );
        }
      }
      
      // Validate issuer (iss claim)
      if (expectedIssuer != null) {
        final issuer = claims['iss'];
        if (issuer != expectedIssuer) {
          return TokenValidationResult(
            isValid: false,
            error: 'Invalid issuer: $issuer',
            claims: claims,
          );
        }
      }
      
      // Validate audience (aud claim)
      if (expectedAudience != null) {
        final audience = claims['aud'];
        // Audience can be a string or list of strings
        final audiences = audience is List ? audience : [audience];
        if (!audiences.contains(expectedAudience)) {
          return TokenValidationResult(
            isValid: false,
            error: 'Invalid audience',
            claims: claims,
          );
        }
      }
      
      return TokenValidationResult(
        isValid: true,
        claims: claims,
      );
    } catch (e) {
      return TokenValidationResult(
        isValid: false,
        error: 'Token validation error: $e',
      );
    }
  }
  
  /// Extracts user information from ID token claims
  static UserInfo extractUserInfo(Map<String, dynamic> claims) {
    return UserInfo(
      userId: claims['sub'],           // Subject - unique user identifier
      email: claims['email'],
      name: claims['name'],
      picture: claims['picture'],
      emailVerified: claims['email_verified'] ?? false,
    );
  }
  
  /// Checks if token needs refresh (expires within 5 minutes)
  static bool needsRefresh(String token) {
    try {
      final claims = decodeToken(token);
      if (!claims.containsKey('exp')) return false;
      
      final exp = claims['exp'] as int;
      final expiration = DateTime.fromMillisecondsSinceEpoch(exp * 1000);
      
      // Refresh if less than 5 minutes until expiration
      return DateTime.now().isAfter(
        expiration.subtract(Duration(minutes: 5)),
      );
    } catch (e) {
      return true; // If we can't parse it, assume it needs refresh
    }
  }
}

/// Result of token validation
class TokenValidationResult {
  final bool isValid;
  final String? error;
  final Map<String, dynamic>? claims;
  
  TokenValidationResult({
    required this.isValid,
    this.error,
    this.claims,
  });
}

/// User information extracted from ID token
class UserInfo {
  final String userId;
  final String? email;
  final String? name;
  final String? picture;
  final bool emailVerified;
  
  UserInfo({
    required this.userId,
    this.email,
    this.name,
    this.picture,
    this.emailVerified = false,
  });
}
```

**Explanation:**

- **JWT Structure**: A JWT consists of three parts separated by dots: `header.payload.signature`. The header contains metadata, the payload contains claims (user data), and the signature ensures integrity.
- **Base64URL Decoding**: JWTs use URL-safe base64 encoding. The code handles the conversion from Base64URL to standard Base64 (replacing `-` with `+`, `_` with `/`) and adds back the padding (`=`) that Base64URL removes.
- **`exp` claim**: "Expiration Time" - Unix timestamp (seconds since epoch) when the token expires. The code converts this to `DateTime` and checks against current time with 60-second leeway for clock differences between client and server.
- **`nbf` claim**: "Not Before" - Timestamp before which the token must not be accepted.
- **`iss` claim**: "Issuer" - Identifies who issued the token (e.g., `https://accounts.google.com`).
- **`sub` claim**: "Subject" - The unique identifier for the user.
- **Clock Skew**: The 60-second buffer accounts for minor time differences between the client's clock and the server's clock, preventing false negatives in validation.

---

## **20.2 Secure Local Storage**

Never store tokens, passwords, or API keys in `SharedPreferences` or plain text files. Use platform-specific secure storage that encrypts data using hardware-backed encryption when available.

### **Implementing Secure Storage**

```dart
// secure_storage_service.dart
// Wrapper around flutter_secure_storage with type safety
// and additional encryption layers

import 'dart:convert';
import 'package:flutter_secure_storage/flutter_secure_storage.dart';
import 'package:flutter/foundation.dart';

/// Service for storing sensitive data securely
/// Uses platform-specific secure storage:
/// - iOS: Keychain (kSecAttrAccessibleWhenUnlockedThisDeviceOnly)
/// - Android: EncryptedSharedPreferences (Android 6+) or KeyStore
class SecureStorageService {
  // Singleton instance to ensure single storage instance
  static final SecureStorageService _instance = SecureStorageService._internal();
  factory SecureStorageService() => _instance;
  SecureStorageService._internal();
  
  // Configuration for secure storage
  // These options control encryption and accessibility
  static const _options = IOSOptions(
    // Data accessible only when device is unlocked
    accessibility: KeychainAccessibility.first_unlock_this_device,
    // Data not included in iCloud backup
    accountName: 'flutter_secure_storage',
  );
  
  static const _androidOptions = AndroidOptions(
    // Use EncryptedSharedPreferences (AES-256 encryption)
    encryptedSharedPreferences: true,
    // Key stored in Android Keystore system
    keyCipherAlgorithm: KeyCipherAlgorithm.RSA_ECB_PKCS1Padding,
    // AES-GCM for symmetric encryption of values
    storageCipherAlgorithm: StorageCipherAlgorithm.AES_GCM_NoPadding,
  );
  
  // The underlying secure storage instance
  final _storage = const FlutterSecureStorage(
    iOptions: _options,
    aOptions: _androidOptions,
  );
  
  // Keys used for storing auth data
  static const _keyAccessToken = 'auth_access_token';
  static const _keyRefreshToken = 'auth_refresh_token';
  static const _keyTokenExpiry = 'auth_token_expiry';
  static const _keyUserId = 'auth_user_id';
  
  /// Stores authentication tokens securely
  /// [accessToken]: Short-lived JWT for API access
  /// [refreshToken]: Long-lived token to obtain new access tokens
  /// [expiresIn]: Token lifetime in seconds
  Future<void> storeAuthTokens({
    required String accessToken,
    String? refreshToken,
    required int expiresIn,
  }) async {
    try {
      // Calculate expiration timestamp
      final expiryTime = DateTime.now().add(Duration(seconds: expiresIn));
      
      // Store all values atomically
      await Future.wait([
        _storage.write(key: _keyAccessToken, value: accessToken),
        if (refreshToken != null)
          _storage.write(key: _keyRefreshToken, value: refreshToken),
        _storage.write(
          key: _keyTokenExpiry, 
          value: expiryTime.millisecondsSinceEpoch.toString(),
        ),
      ]);
      
      debugPrint('Tokens stored securely');
    } catch (e) {
      debugPrint('Error storing tokens: $e');
      throw SecureStorageException('Failed to store tokens: $e');
    }
  }
  
  /// Retrieves the stored access token
  /// Returns null if no token exists or if storage is inaccessible
  Future<String?> getAccessToken() async {
    try {
      return await _storage.read(key: _keyAccessToken);
    } catch (e) {
      debugPrint('Error reading access token: $e');
      return null;
    }
  }
  
  /// Retrieves refresh token for token rotation
  Future<String?> getRefreshToken() async {
    try {
      return await _storage.read(key: _keyRefreshToken);
    } catch (e) {
      debugPrint('Error reading refresh token: $e');
      return null;
    }
  }
  
  /// Checks if stored access token is expired
  Future<bool> isTokenExpired() async {
    try {
      final expiryStr = await _storage.read(key: _keyTokenExpiry);
      if (expiryStr == null) return true;
      
      final expiry = int.parse(expiryStr);
      final expiryDate = DateTime.fromMillisecondsSinceEpoch(expiry);
      
      // Consider expired 5 minutes before actual expiry
      return DateTime.now().isAfter(
        expiryDate.subtract(Duration(minutes: 5)),
      );
    } catch (e) {
      return true; // Assume expired if we can't read the value
    }
  }
  
  /// Clears all authentication data
  /// Call this on logout
  Future<void> clearAuthData() async {
    try {
      await Future.wait([
        _storage.delete(key: _keyAccessToken),
        _storage.delete(key: _keyRefreshToken),
        _storage.delete(key: _keyTokenExpiry),
        _storage.delete(key: _keyUserId),
      ]);
      debugPrint('Auth data cleared');
    } catch (e) {
      debugPrint('Error clearing auth data: $e');
      throw SecureStorageException('Failed to clear auth data: $e');
    }
  }
  
  /// Stores arbitrary encrypted data
  /// Use for sensitive user preferences or cached credentials
  Future<void> storeSecureData(String key, String value) async {
    await _storage.write(key: key, value: value);
  }
  
  /// Reads arbitrary encrypted data
  Future<String?> readSecureData(String key) async {
    return await _storage.read(key: key);
  }
  
  /// Deletes specific key
  Future<void> deleteSecureData(String key) async {
    await _storage.delete(key: key);
  }
  
  /// Securely stores complex objects as JSON
  Future<void> storeObject(String key, Map<String, dynamic> object) async {
    final jsonString = jsonEncode(object);
    await _storage.write(key: key, value: jsonString);
  }
  
  /// Retrieves and decodes stored JSON object
  Future<Map<String, dynamic>?> getObject(String key) async {
    final jsonString = await _storage.read(key: key);
    if (jsonString == null) return null;
    
    try {
      return jsonDecode(jsonString) as Map<String, dynamic>;
    } catch (e) {
      debugPrint('Error decoding stored object: $e');
      return null;
    }
  }
}

/// Custom exception for secure storage errors
class SecureStorageException implements Exception {
  final String message;
  SecureStorageException(this.message);
  
  @override
  String toString() => 'SecureStorageException: $message';
}
```

**Explanation:**

- **Keychain Accessibility**: `first_unlock_this_device` means the data is accessible after the first device unlock following a restart, and persists across backups only for this specific device (not transferable to new devices).
- **EncryptedSharedPreferences**: On Android 6.0+, this uses AES-256 GCM mode encryption. The encryption key is stored in the Android Keystore, which is backed by hardware security module (HSM) when available.
- **Atomic Writes**: `Future.wait` ensures all related data (access token, refresh token, expiry) are written together. If any fails, they all fail, preventing inconsistent state.
- **Clock Skew Handling**: When checking expiration, we subtract 5 minutes to account for network latency and clock differences, refreshing tokens proactively rather than reactively.
- **Exception Handling**: All operations wrap platform-specific exceptions (Keychain access denied, Keystore unavailable) into custom `SecureStorageException` for consistent error handling across platforms.

---

## **20.3 Biometric Authentication**

Biometric authentication (Touch ID/Face ID on iOS, Fingerprint/Face Unlock on Android) provides a convenient and secure way to verify user identity before accessing sensitive data.

### **Implementing Biometric Auth**

```dart
// biometric_auth_service.dart
// Integrates local_auth plugin for fingerprint/face authentication

import 'package:flutter/services.dart';
import 'package:local_auth/local_auth.dart';
import 'package:local_auth_android/local_auth_android.dart';
import 'package:local_auth_ios/local_auth_ios.dart';
import 'package:flutter/foundation.dart';

/// Service handling biometric authentication
/// Supports fingerprint, face recognition, and iris scanning (device dependent)
class BiometricAuthService {
  final LocalAuthentication _localAuth = LocalAuthentication();
  
  /// Checks if device supports biometric authentication
  /// Returns BiometricStatus indicating availability and type
  Future<BiometricStatus> checkBiometricSupport() async {
    try {
      // Check if device has hardware support
      final bool isAvailable = await _localAuth.canCheckBiometrics;
      
      if (!isAvailable) {
        return BiometricStatus.notAvailable;
      }
      
      // Get list of available biometric types
      final List<BiometricType> availableBiometrics = 
          await _localAuth.getAvailableBiometrics();
      
      debugPrint('Available biometrics: $availableBiometrics');
      
      // Determine specific types available
      final bool hasFingerprint = availableBiometrics.contains(
        BiometricType.fingerprint,
      );
      final bool hasFace = availableBiometrics.contains(
        BiometricType.face,
      );
      final bool hasIris = availableBiometrics.contains(
        BiometricType.iris,
      );
      final bool hasStrong = availableBiometrics.contains(
        BiometricType.strong,  // Class 3 biometric (Android)
      );
      final bool hasWeak = availableBiometrics.contains(
        BiometricType.weak,    // Class 2 biometric (Android)
      );
      
      // Check if biometrics are enrolled (set up by user)
      final bool isDeviceSupported = await _localAuth.isDeviceSupported();
      
      if (!isDeviceSupported) {
        return BiometricStatus.notEnrolled;
      }
      
      return BiometricStatus(
        isAvailable: true,
        hasFingerprint: hasFingerprint || hasStrong,
        hasFace: hasFace,
        hasIris: hasIris,
        biometricTypes: availableBiometrics,
      );
    } on PlatformException catch (e) {
      debugPrint('Error checking biometrics: $e');
      return BiometricStatus.error(e.message);
    }
  }
  
  /// Authenticates user using biometrics
  /// 
  /// [localizedReason]: Message shown to user explaining why auth is needed
  /// [useErrorDialogs]: Show system dialogs for errors (e.g., no fingerprints enrolled)
  /// [stickyAuth]: Keep app alive during authentication (don't pause)
  Future<AuthResult> authenticate({
    required String localizedReason,
    bool useErrorDialogs = true,
    bool stickyAuth = false,
    bool sensitiveTransaction = true,
  }) async {
    try {
      // Configure authentication options per platform
      final bool isAuthenticated = await _localAuth.authenticate(
        localizedReason: localizedReason,
        authMessages: const [
          // iOS-specific messages
          IOSAuthMessages(
            lockOut: 'Biometric authentication is disabled. '
                     'Please lock and unlock your screen to enable it.',
            goToSettingsButton: 'Settings',
            goToSettingsDescription: 'Please set up biometric authentication '
                                    'in Settings.',
            cancelButton: 'Cancel',
          ),
          // Android-specific messages
          AndroidAuthMessages(
            signInTitle: 'Biometric Authentication Required',
            cancelButton: 'Cancel',
            goToSettingButton: 'Settings',
            goToSettingDescription: 'Please set up biometric authentication '
                                   'on your device.',
            biometricHint: 'Verify your identity',
            biometricNotRecognized: 'Not recognized, try again',
            biometricRequiredTitle: 'Biometric authentication required',
            deviceCredentialsRequiredTitle: 'Device credential required',
            deviceCredentialsSetupDescription: 'Please set up device credentials',
          ),
        ],
        options: AuthenticationOptions(
          useErrorDialogs: useErrorDialogs,
          stickyAuth: stickyAuth,
          sensitiveTransaction: sensitiveTransaction,
          // On Android, prefer biometric over PIN/pattern
          biometricOnly: false,  // Set true to disallow fallback to PIN
        ),
      );
      
      return AuthResult(
        success: isAuthenticated,
        message: isAuthenticated 
            ? 'Authentication successful' 
            : 'Authentication failed or cancelled',
      );
    } on PlatformException catch (e) {
      debugPrint('Authentication error: $e');
      return AuthResult(
        success: false,
        message: _getErrorMessage(e.code),
        errorCode: e.code,
      );
    }
  }
  
  /// Authenticates with strict biometric only (no fallback to PIN/Pattern)
  /// Use for high-security operations like confirming large transactions
  Future<AuthResult> authenticateWithBiometricOnly({
    required String localizedReason,
  }) async {
    try {
      final bool isAuthenticated = await _localAuth.authenticate(
        localizedReason: localizedReason,
        options: const AuthenticationOptions(
          biometricOnly: true,  // Strict biometric, no device credentials
          useErrorDialogs: true,
          stickyAuth: true,
        ),
      );
      
      return AuthResult(
        success: isAuthenticated,
        message: isAuthenticated 
            ? 'Biometric authentication successful' 
            : 'Authentication failed',
      );
    } on PlatformException catch (e) {
      return AuthResult(
        success: false,
        message: _getErrorMessage(e.code),
        errorCode: e.code,
      );
    }
  }
  
  /// Stop authentication (if in progress)
  Future<bool> stopAuthentication() async {
    return await _localAuth.stopAuthentication();
  }
  
  /// Human-readable error messages for platform error codes
  String _getErrorMessage(String code) {
    switch (code) {
      case 'NotAvailable':
        return 'Biometric authentication is not available on this device';
      case 'NotEnrolled':
        return 'No biometric credentials are enrolled. '
               'Please set up fingerprint or face recognition.';
      case 'PasscodeNotSet':
        return 'Device passcode is not set. Please set a PIN or password.';
      case 'LockedOut':
        return 'Too many failed attempts. Biometric authentication is locked. '
               'Please use device credentials or wait.';
      case 'PermanentlyLockedOut':
        return 'Biometric authentication is permanently locked. '
               'Please use device credentials.';
      default:
        return 'Authentication error: $code';
    }
  }
}

/// Status of biometric hardware/support
class BiometricStatus {
  final bool isAvailable;
  final bool hasFingerprint;
  final bool hasFace;
  final bool hasIris;
  final List<BiometricType> biometricTypes;
  final String? errorMessage;
  
  BiometricStatus({
    required this.isAvailable,
    this.hasFingerprint = false,
    this.hasFace = false,
    this.hasIris = false,
    required this.biometricTypes,
    this.errorMessage,
  });
  
  // Factory constructors for common states
  factory BiometricStatus.notAvailable() => BiometricStatus(
        isAvailable: false,
        biometricTypes: [],
        errorMessage: 'Biometric authentication not available',
      );
      
  factory BiometricStatus.notEnrolled() => BiometricStatus(
        isAvailable: false,
        biometricTypes: [],
        errorMessage: 'No biometrics enrolled',
      );
        
  factory BiometricStatus.error(String? message) => BiometricStatus(
        isAvailable: false,
        biometricTypes: [],
        errorMessage: message,
      );
  
  /// Returns user-friendly description of available biometrics
  String get availableMethods {
    if (!isAvailable) return 'None';
    final methods = <String>[];
    if (hasFingerprint) methods.add('Fingerprint');
    if (hasFace) methods.add('Face Recognition');
    if (hasIris) methods.add('Iris');
    return methods.join(', ');
  }
}

/// Result of authentication attempt
class AuthResult {
  final bool success;
  final String message;
  final String? errorCode;
  
  AuthResult({
    required this.success,
    required this.message,
    this.errorCode,
  });
}
```

**Explanation:**

- **`canCheckBiometrics`**: Checks if the device has biometric hardware (fingerprint sensor, Face ID hardware).
- **`getAvailableBiometrics`**: Returns specific types available: `fingerprint`, `face`, `iris`, `strong` (Android Class 3 - hardware-backed), `weak` (Android Class 2 - software-based).
- **`isDeviceSupported()`**: Checks if biometrics are actually enrolled/configured by the user. A device might have a fingerprint sensor but no fingerprints registered.
- **`stickyAuth`**: When `true`, the authentication dialog stays visible even if the app goes to background (important for banking apps where switching apps might be needed).
- **`sensitiveTransaction`**: On iOS, this determines whether the system shows additional confirmation dialogs for sensitive operations.
- **`biometricOnly`**: When `false`, allows fallback to device PIN/Pattern/Password. When `true`, strictly requires biometric authentication (more secure but less convenient).
- **PlatformException Handling**: Specific error codes like `LockedOut` (too many failed attempts) require specific user guidance (use PIN instead).

### **Secure Flow with Biometric + Token**

```dart
// secure_session_manager.dart
// Combines biometric auth with secure token storage
// for high-security app flows

import 'dart:async';
import 'secure_storage_service.dart';
import 'biometric_auth_service.dart';

/// Manages secure app sessions requiring biometric verification
/// Pattern: Biometric gate -> Secure storage -> API access
class SecureSessionManager {
  final SecureStorageService _secureStorage = SecureStorageService();
  final BiometricAuthService _biometricAuth = BiometricAuthService();
  
  // Cache for current session (memory only, not persisted)
  String? _cachedToken;
  DateTime? _lastAuthTime;
  
  // Session timeout (require re-auth after 5 minutes)
  static const _sessionTimeout = Duration(minutes: 5);
  
  /// Checks if user has valid session or needs to authenticate
  Future<SessionStatus> checkSession() async {
    // Check if we have tokens stored
    final token = await _secureStorage.getAccessToken();
    if (token == null) {
      return SessionStatus.noToken;
    }
    
    // Check if we have a recent valid session in memory
    if (_hasValidSession()) {
      return SessionStatus.authenticated;
    }
    
    // Token exists but session expired, need biometric re-auth
    return SessionStatus.requiresAuthentication;
  }
  
  /// Authenticates user and retrieves API token
  /// 1. Check biometrics
  /// 2. If success, get token from secure storage
  /// 3. Cache token in memory for session duration
  Future<AuthResponse> authenticateAndGetToken() async {
    // Step 1: Verify biometric
    final bioResult = await _biometricAuth.authenticate(
      localizedReason: 'Authenticate to access your secure data',
      sensitiveTransaction: true,
    );
    
    if (!bioResult.success) {
      return AuthResponse.failure(bioResult.message);
    }
    
    // Step 2: Retrieve token from secure storage
    final token = await _secureStorage.getAccessToken();
    if (token == null) {
      return AuthResponse.failure('No stored credentials found');
    }
    
    // Step 3: Check if token is expired
    final isExpired = await _secureStorage.isTokenExpired();
    if (isExpired) {
      // Attempt refresh if we have refresh token
      final refreshSuccess = await _attemptTokenRefresh();
      if (!refreshSuccess) {
        return AuthResponse.failure('Session expired, please login again');
      }
    }
    
    // Step 4: Establish session
    _cachedToken = await _secureStorage.getAccessToken();
    _lastAuthTime = DateTime.now();
    
    return AuthResponse.success(_cachedToken!);
  }
  
  /// Gets API token for HTTP requests
  /// Automatically triggers biometric auth if session expired
  Future<String?> getApiToken() async {
    final status = await checkSession();
    
    switch (status) {
      case SessionStatus.authenticated:
        return _cachedToken;
        
      case SessionStatus.requiresAuthentication:
        final result = await authenticateAndGetToken();
        return result.isSuccess ? result.token : null;
        
      case SessionStatus.noToken:
        return null;
    }
  }
  
  /// Attempts to refresh access token using refresh token
  Future<bool> _attemptTokenRefresh() async {
    final refreshToken = await _secureStorage.getRefreshToken();
    if (refreshToken == null) return false;
    
    try {
      // Call your API to refresh token
      // final newTokens = await apiService.refreshToken(refreshToken);
      // await _secureStorage.storeAuthTokens(...);
      
      // Placeholder for actual implementation
      return true;
    } catch (e) {
      return false;
    }
  }
  
  /// Clears session (logout)
  Future<void> logout() async {
    _cachedToken = null;
    _lastAuthTime = null;
    await _secureStorage.clearAuthData();
  }
  
  /// Checks if we have a valid in-memory session
  bool _hasValidSession() {
    if (_cachedToken == null || _lastAuthTime == null) return false;
    
    return DateTime.now().difference(_lastAuthTime!) < _sessionTimeout;
  }
}

enum SessionStatus {
  authenticated,           // Valid session exists
  requiresAuthentication,  // Token exists but needs biometric verification
  noToken,                // No stored credentials
}

class AuthResponse {
  final bool isSuccess;
  final String? token;
  final String? errorMessage;
  
  AuthResponse._(this.isSuccess, this.token, this.errorMessage);
  
  factory AuthResponse.success(String token) => 
      AuthResponse._(true, token, null);
      
  factory AuthResponse.failure(String message) => 
      AuthResponse._(false, null, message);
}
```

**Explanation:**

- **Memory Caching**: `_cachedToken` stores the token in memory (RAM) only, never in plain text persistent storage. This allows quick access during the session without repeated Keychain/Keystore access overhead.
- **Session Timeout**: `_sessionTimeout` defines how long the app can be used before requiring re-authentication. This balances security (don't keep session indefinitely) with usability (don't ask for fingerprint every 10 seconds).
- **State Machine**: `SessionStatus` manages the three possible states: fully authenticated (cached), requiring biometric (token exists but session expired), or logged out (no token).
- **Token Refresh**: Integrated logic to automatically refresh access tokens when expired, using the securely stored refresh token, before the user notices.

---

## **20.4 Certificate Pinning & Network Security**

Certificate pinning prevents Man-in-the-Middle (MITM) attacks by ensuring your app only communicates with servers presenting specific SSL certificates, not just any valid certificate.

### **Implementing SSL Pinning**

```dart
// http_client_with_pinning.dart
// Configures HTTP client with certificate pinning
// Prevents MITM attacks using rogue certificates

import 'dart:io';
import 'dart:typed_data';
import 'package:http/http.dart' as http;
import 'package:http/io_client.dart';

/// Creates HTTP client with SSL certificate pinning
/// 
/// Certificate pinning ensures app only trusts specific certificates
/// rather than any certificate signed by a trusted CA
class SecureHttpClient {
  /// Creates HTTP client with public key pinning (recommended)
  /// 
  /// [publicKeyPins]: Map of hostnames to list of allowed public key hashes
  /// (Base64-encoded SHA-256 hashes of the SPKI)
  static Future<http.Client> createPinnedClient({
    required Map<String, List<String>> publicKeyPins,
    Duration? timeout,
  }) async {
    
    // Create SecurityContext for custom certificate validation
    final context = SecurityContext(withTrustedRoots: true);
    
    // Create HttpClient with custom certificate callback
    final httpClient = HttpClient(context: context);
    
    // Set connection timeout
    if (timeout != null) {
      httpClient.connectionTimeout = timeout;
    }
    
    // Configure bad certificate callback for pinning validation
    httpClient.badCertificateCallback = 
        (X509Certificate cert, String host, int port) {
      
      // If host not in pin list, use default validation (trust system CA)
      if (!publicKeyPins.containsKey(host)) {
        return true; // Allow system validation to proceed
      }
      
      // Extract public key from certificate
      final publicKey = cert.pem;  // PEM-encoded certificate
      
      // Calculate SHA-256 hash of public key
      final publicKeyHash = _calculatePublicKeyHash(publicKey);
      
      // Check if hash matches any allowed pin
      final allowedPins = publicKeyPins[host] ?? [];
      final isPinned = allowedPins.contains(publicKeyHash);
      
      if (!isPinned) {
        // Log security violation (don't crash in production, but block)
        _logSecurityViolation(host, publicKeyHash, allowedPins);
        return false; // Reject connection
      }
      
      return true; // Accept pinned certificate
    };
    
    // Wrap in IOClient to use with http package interface
    return IOClient(httpClient);
  }
  
  /// Calculates SHA-256 hash of certificate public key
  /// In production, use proper X509 parsing to extract SubjectPublicKeyInfo
  static String _calculatePublicKeyHash(String pemCertificate) {
    // This is a simplified example
    // Production implementation should:
    // 1. Parse PEM to extract SubjectPublicKeyInfo (SPKI)
    // 2. SHA-256 hash the DER-encoded SPKI
    // 3. Base64 encode the hash
    
    // For actual implementation, use x509 package or platform channels
    // to extract the public key properly
    
    // Placeholder implementation
    return '';
  }
  
  /// Logs certificate pinning failures for security monitoring
  static void _logSecurityViolation(
    String host, 
    String receivedHash, 
    List<String> expectedHashes,
  ) {
    // In production, send to crash reporting or security monitoring
    print('SECURITY VIOLATION: Certificate pinning failed for $host');
    print('Received hash: $receivedHash');
    print('Expected hashes: $expectedHashes');
    
    // Consider throwing a specific exception or reporting to backend
  }
  
  /// Alternative: Create client with certificate file pinning
  /// Embeds actual certificate file in app bundle
  static Future<http.Client> createCertFileClient({
    required Map<String, String> certificateAssets,  // host -> asset path
  }) async {
    final context = SecurityContext(withTrustedRoots: false);
    
    // Load certificates from assets
    for (final entry in certificateAssets.entries) {
      final certData = await _loadCertificateAsset(entry.value);
      
      // Set certificate authority for specific host
      context.setTrustedCertificatesBytes(certData);
    }
    
    final httpClient = HttpClient(context: context);
    return IOClient(httpClient);
  }
  
  static Future<Uint8List> _loadCertificateAsset(String path) async {
    // Implementation to load cert from Flutter assets
    // rootBundle.load(path)...
    return Uint8List(0);
  }
}

/// Usage example with Dio package (alternative to http)
/// Dio provides built-in pinning support
class DioPinningConfig {
  static Future<void> configureDioPinning(
    dioInstance, {
    required List<String> allowedFingerprints,
  }) async {
    // Dio certificate callback for pinning
    dioInstance.httpClientAdapter = IOHttpClientAdapter(
      createHttpClient: () {
        final client = HttpClient();
        client.badCertificateCallback = (cert, host, port) {
          // Get certificate SHA-1 or SHA-256 fingerprint
          final fingerprint = cert.sha1;  // or calculate SHA-256
          
          // Compare against allowed pins
          return allowedFingerprints.any((pin) => 
            fingerprint.toString().contains(pin),
          );
        };
        return client;
      },
    );
  }
}
```

**Explanation:**

- **`SecurityContext`**: Dart's `dart:io` class for configuring TLS/SSL settings. `withTrustedRoots: true` includes system CA certificates; `false` creates a "clean room" with only your specified certificates.
- **`badCertificateCallback`**: Called when the server presents a certificate. Return `true` to accept, `false` to reject. This is where pinning logic executes.
- **SPKI (Subject Public Key Info) Pinning**: Instead of pinning the entire certificate (which changes on renewal), pin the hash of the public key. The public key persists across certificate renewals if you keep the same key pair.
- **Backup Pins**: Always include at least one backup pin (e.g., a spare key) in your allowed list. If you lose the primary private key and need to rotate, the backup prevents bricking the app.
- **Logging**: Certificate pinning failures indicate active MITM attacks (or misconfigured servers). Log these aggressively for security auditing.

---

## **20.5 Environment Variables & Secrets Management**

Never hardcode API keys, secrets, or environment-specific URLs in your source code. Use compile-time environment variables and secure storage for runtime secrets.

### **Secure Configuration Management**

```dart
// app_config.dart
// Manages environment-specific configuration
// Uses compile-time environment variables + secure storage

import 'dart:io';
import 'package:flutter/foundation.dart';

/// Application configuration loaded from environment
/// Compile-time variables are safe for non-sensitive config
/// Runtime secrets are loaded from secure storage
class AppConfig {
  // Environment type (dev, staging, prod)
  static const String environment = String.fromEnvironment(
    'ENV',
    defaultValue: 'development',
  );
  
  // Non-sensitive API configuration
  static const String apiBaseUrl = String.fromEnvironment(
    'API_BASE_URL',
    defaultValue: 'https://api-dev.example.com',
  );
  
  // Feature flags from environment
  static const bool enableLogging = bool.fromEnvironment(
    'ENABLE_LOGGING',
    defaultValue: kDebugMode,
  );
  
  // Analytics key (non-sensitive, can be in compile-time env)
  static const String analyticsKey = String.fromEnvironment(
    'ANALYTICS_KEY',
    defaultValue: 'dev-key',
  );
  
  /// Validates that required configuration is present
  static void validate() {
    if (apiBaseUrl.isEmpty) {
      throw StateError('API_BASE_URL not configured');
    }
    
    // Ensure production config in release builds
    if (kReleaseMode && environment == 'development') {
      throw StateError('Release build using dev config');
    }
  }
  
  /// Returns appropriate config based on environment
  static EnvironmentConfig get current {
    switch (environment) {
      case 'production':
        return EnvironmentConfig.production();
      case 'staging':
        return EnvironmentConfig.staging();
      case 'development':
      default:
        return EnvironmentConfig.development();
    }
  }
}

/// Environment-specific configuration
class EnvironmentConfig {
  final String apiUrl;
  final String authDomain;
  final int connectTimeout;
  final int receiveTimeout;
  final bool enableCrashlytics;
  
  EnvironmentConfig({
    required this.apiUrl,
    required this.authDomain,
    required this.connectTimeout,
    required this.receiveTimeout,
    required this.enableCrashlytics,
  });
  
  factory EnvironmentConfig.development() => EnvironmentConfig(
        apiUrl: 'https://api-dev.example.com',
        authDomain: 'dev.auth.example.com',
        connectTimeout: 30000,  // 30 seconds
        receiveTimeout: 30000,
        enableCrashlytics: false,
      );
      
  factory EnvironmentConfig.staging() => EnvironmentConfig(
        apiUrl: 'https://api-staging.example.com',
        authDomain: 'staging.auth.example.com',
        connectTimeout: 20000,
        receiveTimeout: 20000,
        enableCrashlytics: true,
      );
        
  factory EnvironmentConfig.production() => EnvironmentConfig(
        apiUrl: 'https://api.example.com',
        authDomain: 'auth.example.com',
        connectTimeout: 15000,  // Stricter timeouts in prod
        receiveTimeout: 15000,
        enableCrashlytics: true,
      );
}

/// Secrets manager for runtime-sensitive configuration
/// Loads from secure storage or encrypted config files
class SecretsManager {
  static final Map<String, String> _secrets = {};
  static bool _initialized = false;
  
  /// Initialize secrets from secure storage
  /// Call this early in app startup (after secure storage is available)
  static Future<void> initialize() async {
    if (_initialized) return;
    
    // Load from secure storage or encrypted asset
    // final storage = SecureStorageService();
    // _secrets['api_key'] = await storage.readSecureData('api_key');
    
    _initialized = true;
  }
  
  /// Get secret by key
  /// Throws if not initialized or key not found
  static String getSecret(String key) {
    if (!_initialized) {
      throw StateError('SecretsManager not initialized');
    }
    
    final value = _secrets[key];
    if (value == null) {
      throw StateError('Secret $key not found');
    }
    
    return value;
  }
  
  /// Set secret at runtime (e.g., after login)
  static void setSecret(String key, String value) {
    _secrets[key] = value;
  }
  
  /// Clear all secrets (logout)
  static void clear() {
    _secrets.clear();
    _initialized = false;
  }
}

/// Build configuration for different flavors
/// Used by CI/CD pipelines to inject values at build time
/// 
/// Build command example:
/// flutter build apk --dart-define=ENV=production \
///                   --dart-define=API_BASE_URL=https://api.example.com
class BuildConfig {
  /// Returns true if running in debug mode with dev config
  static bool get isDevelopment => 
      !kReleaseMode && AppConfig.environment == 'development';
      
  /// Returns true if running production build
  static bool get isProduction => 
      kReleaseMode && AppConfig.environment == 'production';
      
  /// Security check: Prevent debug features in release
  static void securityCheck() {
    if (kReleaseMode) {
      // Ensure no debug endpoints are configured
      assert(!AppConfig.apiBaseUrl.contains('localhost'));
      assert(!AppConfig.apiBaseUrl.contains('dev'));
      
      // Ensure no test credentials
      assert(!AppConfig.analyticsKey.contains('test'));
    }
  }
}
```

**Explanation:**

- **`String.fromEnvironment`**: Reads values passed at compile-time using `--dart-define`. These are baked into the compiled binary and cannot be changed at runtime, making them suitable for API URLs but not for rotating secrets.
- **`kReleaseMode`**: A constant from `flutter/foundation.dart` that is `true` only in release builds. Used to enforce security policies (no dev configs in production).
- **Environment Flavors**: Different configurations for dev/staging/prod ensure development conveniences (longer timeouts, verbose logging) don't leak into production.
- **SecretsManager**: For runtime secrets (API keys obtained after authentication), use memory-only storage (not compile-time) so they can be cleared on logout and aren't visible in binary inspection.
- **Security Assertions**: The `securityCheck()` method uses `assert()` statements that are stripped from release builds but catch configuration errors during development and testing.

---

## **Chapter Summary**

In this chapter, we covered essential security practices for Flutter applications:

### **Key Takeaways:**

1. **OAuth 2.0 Implementation**: Use PKCE (Proof Key for Code Exchange) for secure authorization flows. Never use implicit flow in mobile apps. Store the code verifier securely and validate state parameters to prevent CSRF.

2. **JWT Handling**: Decode tokens to check expiration (`exp`) and issuer (`iss`) client-side for UX purposes, but always verify signatures server-side. Handle clock skew by adding time buffers.

3. **Secure Storage**: Use `flutter_secure_storage` which leverages iOS Keychain (`kSecAttrAccessibleWhenUnlockedThisDeviceOnly`) and Android EncryptedSharedPreferences (AES-256 with Keystore-backed keys). Never use `SharedPreferences` for tokens.

4. **Biometric Authentication**: Check for hardware availability (`canCheckBiometrics`) and enrollment (`isDeviceSupported`). Use `biometricOnly: true` for high-security operations, `false` for convenience with fallback to PIN. Handle `LockedOut` errors gracefully.

5. **Certificate Pinning**: Pin the SPKI (Subject Public Key Info) hash rather than full certificates. Include backup pins. Log pinning failures as potential MITM attacks. Use `SecurityContext` with custom `badCertificateCallback`.

6. **Environment Management**: Use `--dart-define` for compile-time configuration (API URLs, feature flags). Use Secure Storage for runtime secrets. Validate configuration in release builds to prevent debug endpoints in production.

### **Security Checklist:**

- [ ] No hardcoded API keys or secrets in source code
- [ ] OAuth flow uses PKCE
- [ ] Tokens stored in Keychain/Keystore, not UserDefaults/SharedPreferences
- [ ] Biometric auth implemented for sensitive actions
- [ ] SSL pinning enabled for production APIs
- [ ] Certificate validation failures are logged
- [ ] Session timeouts implemented
- [ ] Auto-logout on token expiration
- [ ] Debug logs disabled in release builds
- [ ] Screenshots disabled for sensitive screens (iOS: `UIApplication.shared.isIdleTimerDisabled`)

### **Next Steps:**

The next chapter (Chapter 21) will cover **GraphQL & WebSockets**, including:
- GraphQL client setup and queries/mutations/subscriptions
- WebSocket connections for real-time data
- Handling connection state and reconnection logic
- Server-Sent Events (SSE) for push notifications

---

**End of Chapter 20**

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. http_requests_and_rest_api.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. graphql_and_websockets.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
